# Optimization showcase #

We will try to use our system to generate some good shell designs

Design considerations:

1. Optimizer picks the shell with the largest diameter available for chosen loader. It provides us maximum DPS for given autoloader size. Though railgun shells are more complicated.
2. We use 1 autoloader with all 4 clips attached. Attaching more loaders will increase DPS, but this value should be balanced with coolder modules.
3. Damage and reload time for bleeder shells are inaccurate. Velocity calculation for bleeder shells is OP.
4. Sometimes optimizer picks a bit smaller diameter than it actually could. It is related to modules with limited length, like fuses or bleeder module. But this error is quite low.


In [1]:
import ftd_calc as FTD
from report import displayTable

This function is used to filter out some results

In [2]:
def filterResult(config):
    if config.get("velocity", 0)  < 50:
        return -1.0
    return config["dps"]

In [3]:
# Number of top variants to be generated
optimizer = FTD.ShellOptimizer(max_modules=8, max_results=4, score_fn=filterResult)
# A list to store results
results = []

batch = 4

Generating shell variants for autoloaders with size 1 and 2. We limit number of shell modules to 8

In [4]:
results += optimizer.calcBestShells(loader_length=1, loaders=2, clipsPerLoader=4, velCharge=0)
results += optimizer.calcBestShells(loader_length=2, loaders=2, clipsPerLoader=4, velCharge=0)

Generating shell variants for the same autoloader size, but adding some rail charge

In [5]:
results += FTD.calcBestShells(1, 20, batch, dict(loaders=2, clipsPerLoader=4, velCharge=1000), filterResult)
results += FTD.calcBestShells(2, 20, batch, dict(loaders=2, clipsPerLoader=4, velCharge=1000), filterResult)

Let's see our results:

In [6]:
displayTable(results)

DPS,DAMAGE,DIAMETER,VELOCITY,PERIOD,BLOCKS,SHELL
1113.03,kinetic=5865:9.1,200,344.61,5.27,36,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder']"
1120.56,kinetic=4217:12.7,142,477.62,3.76,41,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder', 'gunpowder']"
1125.84,kinetic=4943:14.8,166,468.36,4.39,39,"['apcap', 'bsabot', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']"
1150.98,kinetic=5054:11.2,166,423.76,4.39,39,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']"
1618.32,kinetic=24120:9.7,400,366.80,14.90,48,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder']"
1629.27,kinetic=17345:13.5,285,508.37,10.65,56,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder', 'gunpowder']"
1636.95,kinetic=20331:15.8,333,498.51,12.42,53,"['apcap', 'bsabot', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']"
1673.50,kinetic=20785:11.9,333,451.03,12.42,53,"['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']"
2467.00,kinetic=3249:56.0,50,1811.62,1.32,23,"['apcap', 'bsabot', 'bsabot', 'bsabot', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'bleeder', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail']"
2479.38,kinetic=3266:45.1,50,1678.04,1.32,23,"['apcap', 'bsabot', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'bleeder', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail']"


You can check specific shell designs by:

In [14]:
shell_bp = ['apcap', 'bsabot', 'bsabot', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'solid', 'bleeder', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail', 'rail']
FTD.calcCannonData(dict(diameter=0.018, loaders=2, clipsPerLoader=4,velCharge=1000, **FTD.calcBulletStats(shell_bp)))

{'accuracy': 0.019319627325598183,
 'armorC': 2.9185371211899707,
 'blocks': 23,
 'clipsPerLoader': 4,
 'damage': {'kinetic': (1315.428925676838, 142.2276501534383)},
 'diameter': 0.018,
 'dps': 4622.7582656731975,
 'expMod': 0.25,
 'kineticC': 4.7,
 'length': 0.36,
 'loaders': 2,
 'modules': 20,
 'period': 0.28455498861895956,
 'rails': 8,
 'shell': ['apcap',
  'bsabot',
  'bsabot',
  'solid',
  'solid',
  'solid',
  'solid',
  'solid',
  'solid',
  'solid',
  'solid',
  'bleeder',
  'rail',
  'rail',
  'rail',
  'rail',
  'rail',
  'rail',
  'rail',
  'rail'],
 'shellLength': 0.21599999999999997,
 'speedC': 1.8023295065947913,
 'velCharge': 1000,
 'velocity': 4873.251366953594,
 'vr': 4873.251366953594}

In [13]:
shell_bp = ['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder']
FTD.calcCannonData(dict(diameter=0.5, loaders=2, clipsPerLoader=4, **FTD.calcBulletStats(shell_bp)))

{'accuracy': 0.9982683969692436,
 'armorC': 2.6486486486486487,
 'barrel_p': 24.0,
 'blocks': 55,
 'clipsPerLoader': 4,
 'coolers': 8,
 'damage': {'kinetic': (47532.19895732094, 12.39029571026538)},
 'diameter': 0.5,
 'dps': 2083.1225793005874,
 'expMod': 1.0,
 'kineticC': 6.666666666666667,
 'length': 3.0,
 'loaders': 2,
 'modules': 6,
 'period': 22.81776378866767,
 'propellant': 3,
 'shell': ['apcap', 'solid', 'solid', 'gunpowder', 'gunpowder', 'gunpowder'],
 'shellLength': 1.5,
 'speedC': 1.3864864864864865,
 'velocity': 467.79687885695824,
 'vp': 467.79687885695824}